# MLP w TensorFlow

Najpierw musimy zaimportować bibliotekę tensorflow. 


In [3]:
import tensorflow as tf

Następnie musimy określić liczbę wejść i wyjść oraz ustawić liczbę ukrytych neuronów w każdej warstwie

In [4]:
n_inputs = 28*28 # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

Następnie, można użyć węzłów typu placeholder do reprezentowania danych treningowych. 

  * Kształt $X$ jest tylko częściowo zdefiniowany. 
  * Wiemy, że będzie to tensor 2D, z instancjami wzdłuż pierwszego wymiaru i cechami wzdłuż drugiego wymiaru, i wiemy, że liczba funkcji będzie wynosić $28 x 28$ (jedna cecha na piksel), ale nie wiemy jeszcze, ile wystąpi elementów. 
  * Zatem kształt $X$ to (None, n_inputs)
  * Podobnie, wiemy, że $y$ będzie tensorem 1D z jedą kolumną na instancję, ale znowu nie znamy wielkości zbioru treningowego, więc kształt jest (None).

In [5]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")